In [1]:
!pipenv install git+https://github.com/stanfordnlp/dspy.git

Mounted at /content/drive
/content/drive/MyDrive/projects/LLM/AgenticRAG/rag_agents


In [1]:
import importlib
import random
import dspy
import dspy_util as util
importlib.reload(util)
import dspy_util as util
from dspy.teleprompt import BootstrapFewShot

random.seed(42)

llm, rm = util.configure_dspy_v1()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\woshi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
rm("What is Jessica’s relationship with Duke Leto?", k=5)[0]

{'id': 'bb89541c-918c-445f-9d9a-671d5e102add',
 'score': 0.3895341348747989,
 'long_text': ' history."\n"Don\'t be facetious, girl! You know as well as I do what forces surround us. We\'ve a three-point civilization: the Imperial Household balanced against the Federated Great Houses of the Landsraad, and between them, the Guild with its damnable monopoly on interstellar transport. In politics, the tripod is the most unstable of all structures. It\'d be bad enough without the complication of a feudal trade culture which turns its back on most science." \nJessica spoke bitterly: "Chips in the path of the flood -- and this chip here, this is the Duke Leto, and this one\'s his son, and this one\'s --"\n"Oh, shut up, girl. You entered this with full knowledge of the delicate edge you walked." \n" \'I am Bene Gesserit: I exist only to serve,\' " Jessica quoted. \n"Truth." the old woman said. "And all we can hope for now is to prevent this from erupting into general conflagration, to salvage 

In [3]:
llm("What version of OpenAI GPT are you?")

["I am based on OpenAI's GPT-3 model. If you have any questions or need assistance, feel free to ask!"]

In [4]:
openai_llm = util.get_openai_few_shot_prompt()

question = """
Question: Who was the actor who played Han Solo in the 1977 movie Star Wars?
- A. Carrie Fisher
- B. Mark Hamil
- C. Harrison Ford
- D. Peter Mayhew
Correct Answer: """
print(f'Answer: {openai_llm.invoke({'question': question})}')

question = """
Question: What does 6 + 7 sum to?
- A. 13
- B. 90
- C. 700
- D. -10
Correct Answer: """
print(f'Answer: {openai_llm.invoke({'question': question})}')

Answer: C
Answer: A


In [5]:
examples = util.load_dataset()
hard_questions = []
for _ in range(5):
    hard_questions.append(examples.pop(-1))
random.shuffle(examples)
train = examples[:10]
dev = examples[10:]
dev = dev + hard_questions
random.shuffle(dev)

In [13]:
class GenerateAnswer(dspy.Signature):
    """The following are multiple choice questions with answers. Return only the letter of the correct answer. Use only the provided context to answer the question."""

    context  = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField(desc="the question", prefix="Question: ")
    choices  = dspy.InputField(desc="choices to choose answer from", prefix="Choices:")
    answer   = dspy.OutputField(desc="the answer whose value is A, B, C, or D",
                                prefix="Correct Answer: ")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        #self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.generate_answer = dspy.Predict(GenerateAnswer)

    def forward(self, question, choices):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question, choices=choices)
        return dspy.Prediction(context=context, answer=prediction.answer)

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    print(f'Example: {example.question}, Prediction: {pred.answer}')
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Compile!
rag = RAG()

# Ask any question you like to this simple RAG program.
question = "Which planet do the Atreides settle on after leaving Caladan?"
choices = "- A. Earth\n- B. Arrakis\n- C. Sietech Tabr\n- D. Salusa Secundus"
# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = rag(question=question, choices=choices)
# Print the contexts and the answer.
print(f"Question: {question}")
print(f"Choices: {choices.replace('\n',',')}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:50] + '...' for c in pred.context]}")


Question: Which planet do the Atreides settle on after leaving Caladan?
Choices: - A. Earth,- B. Arrakis,- C. Sietech Tabr,- D. Salusa Secundus
Predicted Answer: B
Retrieved Contexts (truncated): ['A beginning is the time for taking the most delica...', ' Great Houses of the Landsraad.\n"A popular man aro...', " You'll ride upon your own two feet without 'thopt..."]


In [14]:
uncompiled_result = util.evaluate(dev, rag)
for ex, pred, score in uncompiled_result[1]:
    if score == 0: 
        print(f'Question: {ex.question}')
        print(f'Choices: {ex.choices.replace('\n',',')}')
        print(f'Answer: {ex.answer}, Prediction: {pred.answer}')
        print('----')


Average Metric: 47 / 54  (87.0): 100%|██████████| 54/54 [00:41<00:00,  1.29it/s]
Question: Who is Tuek?
Choices: - A. An Atreides soldier,- B. Baron Harkonnen's Mentat,- C. A captain of the Sardaukar,- D. An Arrakeen smuggler
Answer: D, Prediction: Correct Answer: A
----
Question: Who is Jessica's cousin?
Choices: - A. Alia,- B. The Baron,- C. Feyd-Rautha,- D. Princess Irulan
Answer: C, Prediction: A
----
Question: Which is the baron' grandson?
Choices: - A. Rabban Harkonnen,- B. Stilgar,- C. Feyd-Rautha Harkonnen,- D. Paul Atreides
Answer: D, Prediction: C
----
Question: Who is Usul's sister
Choices: - A. Jessica,- B. Paul,- C. Alia,- D. Princess Irulan
Answer: C, Prediction: A
----
Question: What is the desert planet that serves as the primary setting of Dune?
Choices: - A. Caladan,- B. Mars,- C. Duncan Idaho,- D. Arrakis
Answer: D, Prediction: D. Arrakis
----
Question: Who is Alia's second cousin's uncle?
Choices: - A. Wanna Marcus,- B. The Baron,- C. Feyd-Rautha,- D. Paul
Answer: B

In [15]:
# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = teleprompter.compile(rag, trainset=train)

  0%|          | 0/10 [00:00<?, ?it/s]

Example: Who is the protagonist of Dune?, Prediction: B


 10%|█         | 1/10 [00:01<00:11,  1.23s/it]

Example: Which of the following is not one of the Great Houses of the Landsraad?, Prediction: A


 20%|██        | 2/10 [00:01<00:07,  1.07it/s]

...

Example: Who is Paul Atreides mother?, Prediction: B


 90%|█████████ | 9/10 [00:06<00:00,  1.54it/s]

Example: What do Fremen do with their dead?, Prediction: D


100%|██████████| 10/10 [00:07<00:00,  1.40it/s]


Bootstrapped 2 full traces after 10 examples in round 0.


Evaluate pipeline.

In [16]:
compiled_result = util.evaluate(dev, compiled_rag)
for ex, pred, score in compiled_result[1]:
    if score == 0: 
        print(f'Question: {ex.question}')
        print(f'Choices: {ex.choices.replace('\n',',')}')
        print(f'Answer: {ex.answer}, Prediction: {pred.answer}')
        print('----')

Average Metric: 53 / 54  (98.1): 100%|██████████| 54/54 [00:46<00:00,  1.17it/s] 
Question: Which is the baron' grandson?
Choices: - A. Rabban Harkonnen,- B. Stilgar,- C. Feyd-Rautha Harkonnen,- D. Paul Atreides
Answer: D, Prediction: C
----


In [19]:
dir(llm)

['__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_get_choice_text',
 '_openai_client',
 'basic_request',
 'copy',
 'history',
 'inspect_history',
 'kwargs',
 'log_usage',
 'model_type',
 'print_green',
 'print_red',
 'provider',
 'request',
 'system_prompt']

In [23]:
llm.inspect_history(1)




The following are multiple choice questions with answers. Return only the letter of the correct answer. Use only the provided context to answer the question.

---

Question: What do Fremen do with their dead?
Choices: - A. Bury them in a cave - B. Cremate them - C. Leave them for the sandworms - D. Render them down for their water
Correct Answer: D

Question: Who is Paul Atreides mother?
Choices: - A. Chani - B. Jessica - C. Liet - D. The Reverend Mother
Correct Answer: B

Question: Who is Liet?
Choices: - A. Paul - B. Stilgar - C. Duke Leto - D. Dr. Kynes
Correct Answer: D

Question: How is Paul related to Feyd-Rautha?
Choices: - A. They are besties - B. They are second cousins - C. They are brothers - D. They aren't related
Correct Answer: B

Question: What is the name of the Fremen garment that conserves water?
Choices: - A. Wellsuit - B. Body shield - C. Stillsuit - D. Jubba cloak
Correct Answer: C

Question: For most of the novel, who does Hawat believe betrayed the Atreides?
C

'\n\n\nThe following are multiple choice questions with answers. Return only the letter of the correct answer. Use only the provided context to answer the question.\n\n---\n\nQuestion: What do Fremen do with their dead?\nChoices: - A. Bury them in a cave - B. Cremate them - C. Leave them for the sandworms - D. Render them down for their water\nCorrect Answer: D\n\nQuestion: Who is Paul Atreides mother?\nChoices: - A. Chani - B. Jessica - C. Liet - D. The Reverend Mother\nCorrect Answer: B\n\nQuestion: Who is Liet?\nChoices: - A. Paul - B. Stilgar - C. Duke Leto - D. Dr. Kynes\nCorrect Answer: D\n\nQuestion: How is Paul related to Feyd-Rautha?\nChoices: - A. They are besties - B. They are second cousins - C. They are brothers - D. They aren\'t related\nCorrect Answer: B\n\nQuestion: What is the name of the Fremen garment that conserves water?\nChoices: - A. Wellsuit - B. Body shield - C. Stillsuit - D. Jubba cloak\nCorrect Answer: C\n\nQuestion: For most of the novel, who does Hawat bel